In [ ]:
import numpy as np
import pandas as pd

Supponiamo di avere determinato le varie rette "vincitrici"; poiché la parte precedente non fa altro che prendere quella che meglio si adatta ai punti dobbiamo verificare se queste soluzioni siano accettabili, cioè in particolare se siano compatibili con il nostro errore sperimentale. Come già detto a causa dell'errore sui tempi non possiamo aspettarci che le rette attraversino esattamente i punti, però se lo scarto (cioè la distanza orizzontale dalla retta) supera il margine concesso dall'errore sperimentale l'evento va scartato in quanto incompatibile con l'errore e pertanto inaccettabile.   

Il problema da risolvere è quindi come calcolare la "distanza orizzontale" retta-punto - distanza che non è quella tradizionalmente definita mediante proiezione ortogonale/prodotto scalare, ma è piuttosto ottenuta proiettando parallelamente all'asse x (infatti l'indeterminazione riguarda le coordinate x, le uniche su cui il drift time calcolato/misurato restituisce informazione). Proiettando in questo modo individuiamo una retta parallela all'asse x e che interseca in un certo punto $P$ la retta ottenuta mediante regressione lineare; la distanza che cerchiamo è allora la differenza in modulo fra $x_i$ e $P$ - avendo indicato con $(x_i,y_i)$ il punto di partenza.  

Per trovare le coordinate del punto di intersezione fra la retta $y=y_i$ e la retta $y=mx+q$ basta imporre $y_i=mx_P+q$, da cui ottengo la formula $$d_x(P,(x_i,y_i)) = |x_i-x_P| = \Bigg|x_i-\frac{1}{m}(y_i-q)\Bigg| $$
Questa formula diverge per $m\to0 $, ma questo significa solo che siccome per $m=0$ le rette sono parallele queste si incontrano all'infinito - risultato corretto ma non rilevante, visto che noi non abbiamo mai traiettorie orizzontali fisicamente accettabili visto come è stato prodotto il fascio di muoni e vista la sensibilità fino a 45-50 gradi rispetto alla verticale del trigger.  
Se invece $m\to +\infty$ (traiettoria verticale) questa formula mi restituisce $d_x=x_i$ - risultato corretto, visto che quando $q$ è fissato ed $m$ diverge la nostra retta diventa l'asse y (ruota attorno all'intercetta fissata).  
Di conseguenza in tutti i casi di nostro interesse ($m\in\mathbb{R}\setminus\{0\}$, $q\in\mathbb{R}$) la formula di cui sopra restituisce il risultato voluto; in particolare dobbiamo calcolare questa $d_x$ per ogni punto associato ad un particolare evento, prendere la massima e imporre che non superi $v_d\Delta t$ (essendo $\Delta t$ l'incertezza sul drift time, cioè $\Delta t\approx 50 \ \text{ns}$).

In [ ]:
#qui scrivo il filtro supponendo che al dataframe siano state aggiunte le colonne M e Q, rappresentanti rispettivamente pendenza e intercetta della retta vincitrice evento per evento. 
#Questo implicherebbe che per ogni valore (non unico in quanto abbiamo più hits per evento) di ORBIT_CNT ed EVENT abbiamo gli stessi M e Q per tutte le hits di uno stesso evento (stesso valore di orbit ed event);
#in caso si modifica per armonizzare con il file trajectories "madre".
#suppongo inoltre per semplicità che ci sia una colonna chiamata X contenente o XR o XL di ciascuna hit (a seconda di quale delle due abbia vinto, chiaramente).

delta_x = 42/(2*390)*(50) #l'indeterminazione sulle x è v_d*delta_t=42/(2*390)*(2*25). Calcolo questo valore fuori perché non ha senso ricalcolarlo volta per volta, è una costante 
def exp_unc_compatib_check(event): #experimental uncertainty compatibility check
    if (np.abs(event.X - (event.Y-event.Q)/event.M)).max() < delta_x : #questa espressione è già vettorizzata: la valuto contemporaneamente per tutte le hits (righe) di un evento e prendo il massimo
        return event #nota: siccome sono float due valori difficilmente saranno esattamente uguali, quindi anche se fisicamente è ragionevole qui non ha senso mettere <=, facciamo un confronto in più
    else : #mettere prima il caso in cui si tiene rispetto a quello in cui si scarta o viceversa non sono necessariamente alternative equivalenti! Infatti i dati sono stati ripuliti molto,
        return None  #quindi mi aspetto che i dati da scartare saranno una minoranza - e allora tanto vale mettere prima la condizione che viene soddisfatta quasi sempre senza bisogno di arrivare all'else.
        #poi ovviamente questo ansatz va messo alla prova..!

In [ ]:
#poi qualcosa che a meno di "armonizzazione" è:
df1 = df.groupby(["ORBIT_CNT","EVENT"]).apply(exp_unc_compatib_check).dropna().reset_index()
df1
#magari confrontando/salvando il numero di eventi prima e dopo l'applicazione del sanity check